<a href="https://colab.research.google.com/github/WISEPLAT/_NN/blob/main/2_%D1%80%D0%B0%D1%81%D0%BF%D0%BE%D0%B7%D0%BD%D0%B0%D0%B5%D0%BC_%D0%B8%D0%B7%D0%BE%D0%B1%D1%80%D0%B0%D0%B6%D0%B5%D0%BD%D0%B8%D1%8F_%D1%81_%D0%BF%D0%BE%D0%BC%D0%BE%D1%89%D1%8C%D1%8E_%D1%81%D0%B2%D0%B5%D1%80%D1%82%D0%BE%D1%87%D0%BD%D1%8B%D1%85_%D1%81%D0%B5%D1%82%D0%B5%D0%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from keras.datasets import cifar10 # subroutines for fetching the CIFAR-10 dataset
from keras.models import Model # basic class for specifying and training a neural network
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.utils import np_utils # utilities for one-hot encoding of ground truth values
import numpy as np

In [ ]:
from keras.regularizers import l2 # L2-regularisation
l2_lambda = 0.0001
# W_regularizer == kernel_regularizer
# border_mode == padding
# init == kernel_initializer

In [ ]:
batch_size = 32 # in each iteration, we consider 32 training examples at once
num_epochs = 200 # we iterate 200 times over the entire training set
kernel_size = 3 # we will use 3x3 kernels throughout
pool_size = 2 # we will use 2x2 pooling throughout
conv_depth_1 = 32 # we will initially have 32 kernels per conv. layer...
conv_depth_2 = 64 # ...switching to 64 after the first pooling layer
drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
hidden_size = 512 # the FC layer will have 512 neurons

In [ ]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data() # fetch CIFAR-10 data

num_train, depth, height, width = X_train.shape # there are 50000 training examples in CIFAR-10 
num_test = X_test.shape[0] # there are 10000 test examples in CIFAR-10
num_classes = np.unique(y_train).shape[0] # there are 10 image classes

X_train = X_train.astype('float32') 
X_test = X_test.astype('float32')
# X_train /= np.max(X_train) # Normalise data to [0, 1] range
# X_test /= np.max(X_train) # Normalise data to [0, 1] range

X_train /= 255 # Normalise data to [0, 1] range
X_test /= 255 # Normalise data to [0, 1] range

Y_train = np_utils.to_categorical(y_train, num_classes) # One-hot encode the labels
Y_test = np_utils.to_categorical(y_test, num_classes) # One-hot encode the labels

In [ ]:
X_train.shape, Y_train.shape

((50000, 32, 32, 3), (50000, 10))

In [ ]:
# version 1
inp = Input(shape=(depth, height, width)) # N.B. depth goes first in Keras!
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', kernel_initializer='he_uniform', kernel_regularizer=l2(l2_lambda), activation='relu')(inp)
conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', kernel_initializer='he_uniform', kernel_regularizer=l2(l2_lambda), activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size), padding='same')(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)
# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Convolution2D(conv_depth_2, kernel_size, kernel_size, padding='same', kernel_initializer='he_uniform', kernel_regularizer=l2(l2_lambda), activation='relu')(drop_1)
conv_4 = Convolution2D(conv_depth_2, kernel_size, kernel_size, padding='same', kernel_initializer='he_uniform', kernel_regularizer=l2(l2_lambda), activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size), padding='same')(conv_4)
drop_2 = Dropout(drop_prob_1)(pool_2)
# Now flatten to 1D, apply FC -> ReLU (with dropout) -> softmax
flat = Flatten()(drop_2)
hidden = Dense(hidden_size, kernel_initializer='he_uniform', kernel_regularizer=l2(l2_lambda), activation='relu')(flat)
drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, kernel_initializer='glorot_uniform', kernel_regularizer=l2(l2_lambda), activation='softmax')(drop_3)

model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers

model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy



In [ ]:
# version 2
inp = Input(shape=(depth, height, width)) # N.B. depth goes first in Keras!
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', kernel_initializer='he_uniform', kernel_regularizer=l2(l2_lambda), activation='relu')(inp)
conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size, padding='same', kernel_initializer='he_uniform', kernel_regularizer=l2(l2_lambda), activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size), padding='same')(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)
# Now flatten to 1D, apply FC -> ReLU (with dropout) -> softmax
flat = Flatten()(drop_1)
hidden = Dense(hidden_size, kernel_initializer='he_uniform', kernel_regularizer=l2(l2_lambda), activation='relu')(flat)
drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, kernel_initializer='glorot_uniform', kernel_regularizer=l2(l2_lambda), activation='softmax')(drop_3)

model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers

model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy

In [ ]:
model.summary()

Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, 32, 32, 3)]       0         
                                                                 
 conv2d_34 (Conv2D)          (None, 11, 11, 32)        896       
                                                                 
 conv2d_35 (Conv2D)          (None, 4, 4, 32)          9248      
                                                                 
 max_pooling2d_17 (MaxPoolin  (None, 2, 2, 32)         0         
 g2D)                                                            
                                                                 
 dropout_27 (Dropout)        (None, 2, 2, 32)          0         
                                                                 
 flatten_10 (Flatten)        (None, 128)               0         
                                                          

In [ ]:
model.fit(X_train, Y_train, # Train the model using the training set...
          batch_size=batch_size, epochs=num_epochs,
          verbose=1, validation_split=0.1) # ...holding out 10% of the data for validation

Epoch 1/200
1407/1407 [==============================] - 14s 9ms/step - loss: 1.9454 - accuracy: 0.3158 - val_loss: 1.7043 - val_accuracy: 0.4100
Epoch 2/200
1407/1407 [==============================] - 13s 9ms/step - loss: 1.7055 - accuracy: 0.4012 - val_loss: 1.5557 - val_accuracy: 0.4596
Epoch 3/200
1407/1407 [==============================] - 14s 10ms/step - loss: 1.6189 - accuracy: 0.4336 - val_loss: 1.5126 - val_accuracy: 0.4648
Epoch 4/200
1407/1407 [==============================] - 14s 10ms/step - loss: 1.5620 - accuracy: 0.4515 - val_loss: 1.4293 - val_accuracy: 0.5092
Epoch 5/200
1407/1407 [==============================] - 13s 10ms/step - loss: 1.5269 - accuracy: 0.4670 - val_loss: 1.4119 - val_accuracy: 0.5138
Epoch 6/200
1407/1407 [==============================] - 13s 10ms/step - loss: 1.4943 - accuracy: 0.4832 - val_loss: 1.3891 - val_accuracy: 0.5212
Epoch 7/200
1407/1407 [==============================] - 13s 9ms/step - loss: 1.4743 - accuracy: 0.4885 - val_loss: 1.36

In [ ]:
model.evaluate(X_test, Y_test, verbose=1) # Evaluate the trained model on the test set!

313/313 [==============================] - 1s 4ms/step - loss: 1.1813 - accuracy: 0.6249


[1.1813328266143799, 0.6248999834060669]